### This notebook reads openssl .c source code and generates new source code
#### References
##### https://www.tensorflow.org/tutorials/sequences/recurrent
##### https://machinelearningmastery.com/text-generation-lstm-recurrent-neural-networks-python-keras/

### Prerequisites install tensorflow and keras
#### pip install tensorflow
#### pip install keras

In [1]:
import numpy
import os
import sys
import time

from keras.models import Sequential 
from keras.layers import Activation 
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.utils import np_utils

C:\Users\user\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


### Tune these parameters

In [2]:
SEQ_LENGTH = 100
EPOCHS = 10
BATCH_SIZE = 128

### Preprocessing 
#### read all files and concatenate into a single file train.txt
#### Add tags "<start"> and "< eof >"

In [3]:
PWD=os.getcwd()
print("PWD=" + str(PWD))
train_filename = PWD+"./train.txt"

readSize = 0

if (os.path.isfile(train_filename)):
    os.remove(train_filename)
    
# read every 3rd file otherwise getting out of memory errors
i = 0
with open(train_filename, "w") as a:
    for path, subdirs, files in os.walk(PWD+"/openssl-master"):
        for filename in files:
            if (filename.endswith(".c") and (i%3 == 0)):
                #print("Reading "+ filename)
                f = os.path.join(path, filename)
                readSize += os.path.getsize(f)
                currfile = open(f).read()
                a.write("<start>")
                a.write(currfile)
                a.write("<eof>")
                i=i+1
                
print("readSize="+str(readSize))

PWD=C:\Users\user\Desktop\experiments
readSize=73702


### Open "train.txt" convert into lower case and sort them 


In [4]:
raw_text = open(train_filename).read().lower()
INPUT_FILE_LEN = len(raw_text)

chars = sorted(list(set(raw_text)))
VOCAB_LENGTH = len(chars)

print ("Length of file: "+ str(INPUT_FILE_LEN))
print ("Total Vocab length (unique chars in input) : "+ str(VOCAB_LENGTH))

Length of file: 73714
Total Vocab length (unique chars in input) : 67


### create mapping of unique chars to integers, and a reverse mapping


In [5]:
char_to_int = dict((c, i) for i, c in enumerate(chars))
int_to_char = dict((i, c) for i, c in enumerate(chars))

print(char_to_int)
print(int_to_char)

{'\n': 0, ' ': 1, '!': 2, '"': 3, '#': 4, '%': 5, '&': 6, "'": 7, '(': 8, ')': 9, '*': 10, '+': 11, ',': 12, '-': 13, '.': 14, '/': 15, '0': 16, '1': 17, '2': 18, '3': 19, '4': 20, '5': 21, '6': 22, '7': 23, '8': 24, '9': 25, ':': 26, ';': 27, '<': 28, '=': 29, '>': 30, '?': 31, '[': 32, '\\': 33, ']': 34, '^': 35, '_': 36, 'a': 37, 'b': 38, 'c': 39, 'd': 40, 'e': 41, 'f': 42, 'g': 43, 'h': 44, 'i': 45, 'j': 46, 'k': 47, 'l': 48, 'm': 49, 'n': 50, 'o': 51, 'p': 52, 'q': 53, 'r': 54, 's': 55, 't': 56, 'u': 57, 'v': 58, 'w': 59, 'x': 60, 'y': 61, 'z': 62, '{': 63, '|': 64, '}': 65, '~': 66}
{0: '\n', 1: ' ', 2: '!', 3: '"', 4: '#', 5: '%', 6: '&', 7: "'", 8: '(', 9: ')', 10: '*', 11: '+', 12: ',', 13: '-', 14: '.', 15: '/', 16: '0', 17: '1', 18: '2', 19: '3', 20: '4', 21: '5', 22: '6', 23: '7', 24: '8', 25: '9', 26: ':', 27: ';', 28: '<', 29: '=', 30: '>', 31: '?', 32: '[', 33: '\\', 34: ']', 35: '^', 36: '_', 37: 'a', 38: 'b', 39: 'c', 40: 'd', 41: 'e', 42: 'f', 43: 'g', 44: 'h', 45: 'i

In [6]:
dataX = []
dataY = []
for i in range(0, INPUT_FILE_LEN - SEQ_LENGTH, 1):
    seq_in = raw_text[i:i + SEQ_LENGTH]
    seq_out = raw_text[i + SEQ_LENGTH]
    dataX.append([char_to_int[char] for char in seq_in])
    dataY.append(char_to_int[seq_out])

samples = len(dataX)
print( "Total samples: "+ str(samples))

# reshape X to be [samples, time steps, features]
X = numpy.reshape(dataX, (samples, SEQ_LENGTH, 1))

# normalize
X = X / float(VOCAB_LENGTH)

# one hot encode the output variable
y = np_utils.to_categorical(dataY)
print("X.shape=" + str(X.shape))
print("y.shape=" + str(y.shape))

Total samples: 73614
X.shape=(73614, 100, 1)
y.shape=(73614, 67)


### create model with 2 LSTM Layers with dropout 0.2, 1 Dense layer with softmax


In [7]:
model = Sequential()
model.add(LSTM(256, input_shape=(X.shape[1], X.shape[2]), return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(256))
model.add(Dropout(0.2)) # 0.5
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam') 

### Visualize the model

In [8]:
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 100, 256)          264192    
_________________________________________________________________
dropout_1 (Dropout)          (None, 100, 256)          0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 256)               525312    
_________________________________________________________________
dropout_2 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 67)                17219     
Total params: 806,723
Trainable params: 806,723
Non-trainable params: 0
_________________________________________________________________
None


### Fit the model

In [9]:
histroy = model.fit(X, y, epochs=EPOCHS, batch_size=BATCH_SIZE, verbose=1)

Epoch 1/10
73614/73614 [==============================] - 193s 3ms/step - loss: 2.9715
Epoch 2/10
73614/73614 [==============================] - 193s 3ms/step - loss: 2.6835
Epoch 3/10
73614/73614 [==============================] - 196s 3ms/step - loss: 2.4470
Epoch 4/10
73614/73614 [==============================] - 197s 3ms/step - loss: 2.2116
Epoch 5/10
73614/73614 [==============================] - 195s 3ms/step - loss: 2.0355
Epoch 6/10
73614/73614 [==============================] - 194s 3ms/step - loss: 1.8932
Epoch 7/10
73614/73614 [==============================] - 194s 3ms/step - loss: 1.7874
Epoch 8/10
73614/73614 [==============================] - 198s 3ms/step - loss: 1.6939
Epoch 9/10
73614/73614 [==============================] - 195s 3ms/step - loss: 1.6186
Epoch 10/10
73614/73614 [==============================] - 195s 3ms/step - loss: 1.5495


### Plot Model Loss 

In [10]:
import matplotlib.pyplot as plt 
# list all data in history
print(histroy.history.keys())

# summarize history for loss
plt.plot(histroy.history['loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.show()

dict_keys(['loss'])


<Figure size 640x480 with 1 Axes>

### Generate the code

In [19]:
start = numpy.random.randint(0, len(dataX)-1)
pattern = dataX[start]
print ("input code starts with: [", ''.join([int_to_char[value] for value in pattern]), "]")
# generate characters
for i in range(500):
    x = numpy.reshape(pattern, (1, len(pattern), 1))
    x = x / float(VOCAB_LENGTH)
    prediction = model.predict(x, verbose=0)
    index = numpy.argmax(prediction)
    result = int_to_char[index]
    seq_in = [int_to_char[value] for value in pattern]
    sys.stdout.write(result)
    pattern.append(index)
    pattern = pattern[1:len(pattern)]
print("\n#####.")

input code starts with: [ ut, db->db);
    bio_free(out);
    if (j <= 0)
        goto err;

    out = bio_new_file(buf[1], "w ]
");
    if (reenem)
        return 0;
    return ret;
}

ent set_ser_cdrt(const char *frl,
innst char *arg,
                                                                                                                                                                                                                                                                                                                                                                                                        
#####.
